<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/micro_base_help_tapt_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This TAPT pretrains a base adapter on the micro amazon helpfulness dataset.

# Settings
* LR of 0.0001
* No warmup
* Set batch size to 21, gradient_accumulation_steps=2, ~6 reviews per row = 252 batch size (paper has 256)
* Base
* torch_compile=True (Because no adapter)
* early_stopping_patience=10
* 100 epoch




# Setup

In [1]:
from time import time
start_time = time()

In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 36.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-no

In [3]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_MICRO_helpfulness_dataset_condensed"

dataset = load_dataset(dataset_name)

print(dataset_name)

Generating train split:   0%|          | 0/1695 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1694 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1697 [00:00<?, ? examples/s]

BigTMiami/amazon_MICRO_helpfulness_dataset_condensed


In [4]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="micro_base_help_tapt_pretrain_model",
    overwrite_output_dir=True,
    learning_rate=0.0001,
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1, # Saves latest 2 checkpoints
    torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [8]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [14]:
train_results = trainer.train()
print(train_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,1.910900,1.684880
2,1.741100,1.633343
4,1.633700,1.613704
6,1.599700,1.593081
8,1.541600,1.599447
10,1.507900,1.593824
12,1.471800,1.594711
14,1.436900,1.587873
16,1.401700,1.599981
18,1.360200,1.584609


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=891, training_loss=1.5052021553219368, metrics={'train_runtime': 728.3873, 'train_samples_per_second': 232.706, 'train_steps_per_second': 5.492, 'total_flos': 9817169126307840.0, 'train_loss': 1.5052021553219368, 'epoch': 22.0})


In [15]:
eval_results = trainer.evaluate(dataset["test"])
print(eval_results)

{'eval_loss': 1.591552495956421, 'eval_runtime': 7.4301, 'eval_samples_per_second': 227.992, 'eval_steps_per_second': 10.902, 'epoch': 22.0}


In [16]:
trainer.push_to_hub()

events.out.tfevents.1713605609.3e9d77e32d34.3673.1:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1713606345.3e9d77e32d34.3673.2:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/micro_base_help_tapt_pretrain_model/commit/eff1d9badf30ee985d04d0398850c322088f8f19', commit_message='End of training', commit_description='', oid='eff1d9badf30ee985d04d0398850c322088f8f19', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

Total Time: 16 minutes OR 973 seconds


In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()